# Copyright 2022 Cognite AS
## Authentication

- Creating CogniteClient using different methods
  - Interactive login
  - Using Device code
  - Using clientID &amp; client secret
- Checking the login status

You can authenticate the Python SDK with Azure AD by using a token retrieved when a user authenticates or with a static client secret for long-running jobs like extractors or calculations.

### Prerequisites

* Install the Microsoft Authentication Library (MSAL) for Python.

In [ ]:
#! pip install msal
#! poetry add msal

* You need to specify the values for the following configuration parameters:
 * `Tenant ID` - the ID of the Azure AD tenant where the user is registered.
 * `Client ID` - the ID of the application in Azure AD.
 * `Cluster` - the cluster where your CDF project is installed. For example, api and westeurope-1.
 * `CDF project` - the name of the CDF project.

If you don't know which values to use for these variables, contact your CDF administrator or Cognite Support.

You can directly set the values for these parameters here or read as environment variables or from file.

In [1]:
TENANT_ID="48d5043c-cf70-4c49-881c-c638f5796997"
CLIENT_ID="fab52bb5-9de2-4f9e-aefa-712da4b5fe00"
CDF_CLUSTER="westeurope-1"
COGNITE_PROJECT="ds-basics"

Also set the following derived variables, which will be used for obtaining token

In [2]:
BASE_URL = f"https://{CDF_CLUSTER}.cognitedata.com"
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]
TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"
AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000

### Authenticate with user credentials
You can authenticate the Python SDK with Azure AD by using a token retrieved with user credentials.

#### Interactive Login
You can get the token by letting the user sign in interactively via a browser and use the authenticate with interactive login and token refresh flow to access CDF when you're running short-term scripts or using Jupyter.

In [3]:
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import Token
from msal import PublicClientApplication

In [4]:
def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds

In [5]:
creds = authenticate_azure()

In [6]:


# Configuration object for the client
#     Args:
#         client_name (str): A user-defined name for the client. Used to identify number of unique applications/scripts
#             running on top of CDF.
#         project (str): Project. Defaults to project of given API key.
#         credentials (CredentialProvider): Credentials. e.g. APIKey, Token, ClientCredentials.
#         api_subversion (str): API subversion
#         base_url (str): Base url to send requests to. Defaults to "https://api.cognitedata.com"
#         max_workers (int): Max number of workers to spawn when parallelizing data fetching. Defaults to 10.
#         headers (Dict): Additional headers to add to all requests.
#         timeout (int): Timeout on requests sent to the api. Defaults to 30 seconds.
#         file_transfer_timeout (int): Timeout on file upload/download requests. Defaults to 600 seconds.
#         debug (bool): Configures logger to log extra request details to stderr.

cnf = ClientConfig(client_name="my-special-client", project=COGNITE_PROJECT, credentials=Token(creds["access_token"]), base_url=BASE_URL)

/tmp/ipykernel_16278/625658624.py:15: UserWarning: You are using version 4.2.1 of the SDK, however version 4.11.3 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  cnf = ClientConfig(client_name="my-special-client", project=COGNITE_PROJECT, credentials=Token(creds["access_token"]), base_url=BASE_URL)


In [7]:
client = CogniteClient(cnf)

In [8]:
client.iam.token.inspect()

{
    "subject": "-pQT8hiWyTSnAnf3K8dke40Meg1Es9jMU-Gnk-3Bow8",
    "projects": [
        {
            "url_name": "learn-to-demo-cdf",
            "groups": []
        },
        {
            "url_name": "infield-training",
            "groups": []
        },
        {
            "url_name": "learn-cdf",
            "groups": [
                931093191770988,
                4917973121886595,
                7788679289499029
            ]
        },
        {
            "url_name": "learn",
            "groups": [
                2770317677838154,
                3786527027010710,
                4951036444205202
            ]
        },
        {
            "url_name": "cognite-charts",
            "groups": [
                3769382289164156
            ]
        },
        {
            "url_name": "ds-basics",
            "groups": [
                5363027718561933,
                7150266185177628
            ]
        },
        {
            "url_name": "ds-cognitefuncti

#### Using Device Code
If a browser is not available, for example, if you are logged into a terminal, you can use the authenticate with user credentials and device code flow.

In [10]:
def authenticate_device_code():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    device_flow = app.initiate_device_flow(scopes=SCOPES)
    print(device_flow["message"])  # print device code to screen
    creds = app.acquire_token_by_device_flow(flow=device_flow)

    return creds

In [11]:
creds = authenticate_device_code()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CS3UZEMWK to authenticate.


In [12]:
# Configuration object for the client
#     Args:
#         client_name (str): A user-defined name for the client. Used to identify number of unique applications/scripts
#             running on top of CDF.
#         project (str): Project. Defaults to project of given API key.
#         credentials (CredentialProvider): Credentials. e.g. APIKey, Token, ClientCredentials.
#         api_subversion (str): API subversion
#         base_url (str): Base url to send requests to. Defaults to "https://api.cognitedata.com"
#         max_workers (int): Max number of workers to spawn when parallelizing data fetching. Defaults to 10.
#         headers (Dict): Additional headers to add to all requests.
#         timeout (int): Timeout on requests sent to the api. Defaults to 30 seconds.
#         file_transfer_timeout (int): Timeout on file upload/download requests. Defaults to 600 seconds.
#         debug (bool): Configures logger to log extra request details to stderr.

cnf = ClientConfig(client_name="my-special-client", project=COGNITE_PROJECT, credentials=Token(creds["access_token"]), base_url=BASE_URL)

/tmp/ipykernel_16278/3254481970.py:15: UserWarning: You are using version 4.2.1 of the SDK, however version 4.11.3 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  cnf = ClientConfig(client_name="my-special-client", project=COGNITE_PROJECT, credentials=Token(creds["access_token"]), base_url=BASE_URL)


In [13]:
client = CogniteClient(cnf)

In [14]:
client.login.status()

{
    "user": "-pQT8hiWyTSnAnf3K8dke40Meg1Es9jMU-Gnk-3Bow8",
    "project": "",
    "project_id": -1,
    "logged_in": true
}

### Authenticate with client secret

In [ ]:
from cognite.client.credentials import OAuthClientCredentials

Make Sure, You've created an environment variable CLIENT_SECRET with the value of the client secret obtained from admin.

In [ ]:
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
# Obtain the Environment Variables from .env file
dotenv_path = Path(".env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
import os
CLIENT_SECRET = os.getenv("CLIENT_SECRET")  # store secret in env variable

In [ ]:
# print(CLIENT_SECRET)

Or you can also use getpass to enter the value in notebook interactively. (Not recommended)

In [ ]:
#from getpass import getpass
#CLIENT_SECRET = getpass("Enter the Client Secret: ")  # Enter the client secret interactively here

In [ ]:
creds=OAuthClientCredentials(
    token_url=TOKEN_URL, 
    client_id= CLIENT_ID, 
    scopes= SCOPES, 
    client_secret= CLIENT_SECRET
    )
cnf = ClientConfig(client_name="my-special-client", project=COGNITE_PROJECT, credentials=creds)

In [ ]:
client = CogniteClient(cnf)

### Check the login status

In [ ]:
client.login.status()

### Saving the Authentication Code in Script for Later Usage

Let's define the function for each authentication method.

In [ ]:
def interactive_client():
    """Function to Create the Cognite Client, using Interactive Login method
    """
    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    cnf = ClientConfig(
        client_name="my-special-client", 
        project=COGNITE_PROJECT, 
        credentials=Token(creds["access_token"]), 
        base_url=BASE_URL
        )
    client = CogniteClient(cnf)
    return client

In [ ]:
def device_code_client():
    """Function to Create the Cognite Client, using Device code method
    """
    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
    device_flow = app.initiate_device_flow(scopes=SCOPES)
    print(device_flow["message"])  # print device code to screen
    creds = app.acquire_token_by_device_flow(flow=device_flow)
    cnf = ClientConfig(
        client_name="my-special-client", 
        project=COGNITE_PROJECT, 
        credentials=Token(creds["access_token"]), 
        base_url=BASE_URL
        )
    client = CogniteClient(cnf)
    return client

In [ ]:

def client_secret_client():
    """Function to Create the Cognite Client, using Credentials (e.g. ClientID, Client secret)
    """
    creds = OAuthClientCredentials(
    token_url=TOKEN_URL, 
    client_id=CLIENT_ID, 
    client_secret=CLIENT_SECRET, 
    scopes=SCOPES
    )

    cnf = ClientConfig(
      client_name="custom-client-name", 
      project=COGNITE_PROJECT, 
      credentials=creds,
      base_url=BASE_URL
      )
    client = CogniteClient(cnf)
    return client 

Finally create a wrapper function to create the cognite client, which takes method as the argument.

In [ ]:
def create_cognite_client(method="interactive-login") -> CogniteClient:
    """Function to Create the Client

    Args:
        method (str, optional): One of the methods ("interactive-login","device-code","client-secret"). Defaults to "interactive-login".

    Returns:
        CogniteClient: CogniteClient to be used to access Cognite Data Fusion.
    """
    if method=="interactive-login":
        client = interactive_client()
    elif method=="device-code":
        client = device_code_client()
    elif method=="client-secret":
        client = client_secret_client()
    else:
        client = None
        print("Client couldn't be created. Specify one of these methods :interactive-login, device-code, client-secret")
    return client


Finally let's test these methods

In [ ]:
c = create_cognite_client(method="interactive-login")

In [ ]:
c = create_cognite_client(method="device-code")

In [ ]:
c = create_cognite_client(method="client-secret")

## Check Login Status

In [ ]:
c.login.status()